<a href="https://colab.research.google.com/github/zaidalyafeai/AraMeter/blob/master/AraMeter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pyarabic

     |████████████████████████████████| 112kB 6.7MB/s 
  Created wheel for pyarabic: filename=PyArabic-0.6.7-cp36-none-any.whl size=108603 sha256=f3fb95e68288f22ca449426203bc450892af93b6fb1625538a3d5c25d784d14f
  Stored in directory: /root/.cache/pip/wheels/c6/50/b1/4df7f705f36e91360ab04416dbf1017084698d30a7a3645b5e
Successfully built pyarabic


We use a product review dataset

In [0]:
!wget 'https://raw.githubusercontent.com/zaidalyafeai/ARBML/master/datasets/Poem Meters/baits.zip'
!unzip baits.zip

--2020-05-19 11:49:56--  https://raw.githubusercontent.com/zaidalyafeai/ARBML/master/datasets/Poem%20Meters/baits.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2267882 (2.2M) [application/zip]
Saving to: ‘baits.zip’

baits.zip           100%[===================>]   2.16M  --.-KB/s    in 0.1s    

2020-05-19 11:49:57 (21.7 MB/s) - ‘baits.zip’ saved [2267882/2267882]

Archive:  baits.zip
   creating: final_baits/
  inflating: final_baits/train.txt   
  inflating: final_baits/labels.txt  
  inflating: final_baits/test.txt    


## Imports

In [0]:
import tensorflow as tf
import numpy as np
import os
import time
import glob
from random import shuffle
from pyarabic import araby
from tensorflow.keras.layers import GRU, Embedding, Dense, Input, Dropout, Bidirectional, BatchNormalization, Flatten, Reshape
from tensorflow.keras.models import Sequential
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [0]:
with open('final_baits/labels.txt', 'r') as f:
  label2name = f.readlines()
  label2name = [name.replace('\n', '') for name in label2name]

## Read the Dataset

preprocess a review by removing special characters and long spaces

In [0]:
# Read, then decode for py2 compat.
def extract_data(path, thresh = 70, on_shatrs = False):
  global vocab
  
  text = ""
  
  X = []
  y = []
    
  t = open(path, 'r').read()
  t = araby.strip_tashkeel(t)
  
  # remove some exteranous chars 
  execluded = '!()*-ـ.:=o[]«»;؛,،~?؟\u200f\ufeffـ'
  out = ""
  
  for char in t:
    if char not in execluded:
      out += char
      
  text += out
  baits = out.split('\n')
  for line in baits:
    if len(line) <= 1:
      continue
    label, bait = line.split(' ', 1)
    label = int(label)

    bait  = bait.strip()
    if on_shatrs:
      shatrs = bait.split('#')
      for shatr in shatrs:
        X.append(shatr.strip())
        y.append(label)
    else:
      X.append(bait.strip())
      y.append(label)
  
  #create the vocab 
  vocab = sorted(set(' '.join(X)))  
  
  #shuffle the data 
  X, y = shuffle(X, y)
  return X, y

In [0]:
X, y = extract_data("final_baits/train.txt", on_shatrs=False)

In [0]:
for i in range(5):
  print(X[i], ' ', label2name[y[i]])

وإلى كم شرائي بالجر منه # وانصرافي بخاطر مكسور   khafeef
قل للمعنف عن هواه بجهله # زد من ملامك يا عذول وعنف   kamel
سرى تطوى به الفلوات طيا # ونعم الذخر في يوم النشور   wafer
لما غدت صعدته منآدة # ثقفتم المائل من صعاده   rajaz
مولى ملوك الأنام قاطبة # فيمم البحر وامض عن قلبه   munsareh


## Create Sequences
Create sequences by using the most repeated 500 words

## Create Numpy Arrays

In [0]:
X_train, X_valid , y_train, y_valid = train_test_split(X, y, test_size = 0.15, random_state = 41)

In [0]:
# Creating a mapping from unique characters to indices
char2idx = {u:i+1 for i, u in enumerate(vocab)}

def to_sequences(X):
  X = [[char2idx[char] for char in line] for line in X]
  X = pad_sequences(X, padding='post', value=0, maxlen = 100)
  return X
 
X_train = to_sequences(X_train)
X_valid = to_sequences(X_valid)

y_train = np.array(y_train)
y_valid = np.array(y_valid)

## Create the model

In [0]:
model = Sequential()
model.add(Input((100,)))
model.add(Embedding(len(char2idx)+1, 256))
model.add(Bidirectional(GRU(units = 256, return_sequences=True)))
model.add(Bidirectional(GRU(units = 256, return_sequences=True)))
model.add(Bidirectional(GRU(units = 256)))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(len(label2name), activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 256)          9984      
_________________________________________________________________
bidirectional (Bidirectional (None, 100, 512)          789504    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 512)          1182720   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 512)               1182720   
_________________________________________________________________
dense (Dense)                (None, 128)               65664     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 14)                1

In [0]:
model(tf.zeros((10, 100))).shape

TensorShape([10, 14])

## Train the model

In [0]:
callbacks = [tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_delta=0.0001, min_lr=0.0001)]
callbacks += [tf.keras.callbacks.ModelCheckpoint('full_verse.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')]

In [0]:
model.fit(X_train, y_train, validation_data= (X_valid, y_valid), epochs = 15, batch_size= 128, shuffle = True, callbacks=callbacks)

In [0]:
model = tf.keras.models.load_model('full_verse.h5')

## Tests

In [0]:
def classify(sentence):
#   sentence = process_review(sentence)
  sentence = araby.strip_tashkeel(sentence)
  sequence = [char2idx[char] for char in sentence]
  sequence = pad_sequences([sequence], maxlen = X_train.shape[1], padding='post', value=0)

  pred = model.predict(sequence)[0]
  print(label2name[np.argmax(pred, 0).astype('int')], np.max(pred))
  

In [0]:
classify("ما تردون على هذا المحب # دائبا يشكو إليكم في الكتب")
classify("ولد الهدى فالكائنات ضياء # وفم الزمان تبسم وسناء")
classify(" لك يا منازل في القلوب منازل # أقفرت أنت وهن منك أواهل")
classify("ومن لم يمت بالسيف مات بغيره # تعددت الأسباب والموت واحد")
classify("أنا النبي لا كذب # أنا ابن عبد المطلب")
classify("هذه دراهم اقفرت # أم ربور محتها الدهور")
classify("هزجنا في بواديكم # فأجزلتم عطايانا")
classify("بحر سريع ماله ساحل # مستفعلن مستفعلن فاعلن")
classify("مَا مَضَى فَاتَ وَالْمُؤَمَّلُ غَيْبٌ # وَلَكَ السَّاعَةُ الَّتِيْ أَنْتَ فِيْهَا")
classify("يا ليلُ الصبّ متى غدهُ # أقيامُ الساعة موعدهُ")

ramal 0.9999938
kamel 0.99530077
kamel 0.99872977
taweel 0.9999771
hazaj 0.5043862
madeed 0.527348
hazaj 0.9807406
saree 0.9985551
khafeef 0.9998852
mutadarak 0.99972767


In [0]:
!zip model.zip full_verse.h5

  adding: full_verse.h5 (deflated 7%)


In [0]:
!ls -l

total 75404
-rw-r--r-- 1 root root  2267882 May 19 11:49 baits.zip
drwxr-xr-x 2 root root     4096 Aug  1  2019 final_baits
-rw-r--r-- 1 root root 38894480 May 19 11:58 full_verse.h5
-rw-r--r-- 1 root root 36038534 May 19 12:09 model.zip
drwxr-xr-x 1 root root     4096 May 13 16:29 sample_data
